In [3]:
import mido
import numpy as np
import matplotlib.pyplot as plt
import PreProcessor as pp
import gc
from keras.models import Model
from tensorflow.keras.layers import Input, SimpleRNN, LSTM, GRU, Dense, Lambda, Softmax, Activation, concatenate
from keras import optimizers
import tensorflow as tf

In [2]:
dataset = pp.load_dataset("adl-piano-midi")
ClassicSongs = pp.files_to_songs(dataset["Classical"])

channel_to_ind, ind_to_channel, note_to_ind, ind_to_note, velocity_to_ind, ind_to_velocity = pp.dicts_from_songs(ClassicSongs)
time_range = pp.ranges_from_songs(ClassicSongs)

n_Channels = len(channel_to_ind)
n_Notes = len(note_to_ind)
n_Velocities = len(velocity_to_ind)

print("\nNumber of channels   :",n_Channels,"\nNumber of notes      :",n_Notes,"\nNumber of velocities :",n_Velocities,"\nTime range           :",time_range[0],time_range[1])

Couldn't load : adl-piano-midi\Classical\Classical\Alexander Borodin\Nocturne.mid
Couldn't load : adl-piano-midi\Classical\Classical\Antonio Salieri\Invention.mid
Couldn't load : adl-piano-midi\Classical\Classical\Bela Bartok\Andante.mid
Couldn't load : adl-piano-midi\Classical\Classical\Bela Bartok\Romanian Folk Dance.mid
Couldn't load : adl-piano-midi\Classical\Classical\Claude Debussy\Clair De Lune.mid
Couldn't load : adl-piano-midi\Classical\Classical\Franz Schubert\Impromptu in C-Moll.mid
Couldn't load : adl-piano-midi\Classical\Classical\Franz Schubert\Impromptu in Ges-dur.mid
Couldn't load : adl-piano-midi\Classical\Classical\Franz Schubert\Trois Marches militaires 3.mid
Couldn't load : adl-piano-midi\Classical\Classical\Frederic Chopin\Prelude Op 28 No7.mid
Couldn't load : adl-piano-midi\Classical\Classical\Giacomo Puccini\O Mio Babibino Caro.mid
Couldn't load : adl-piano-midi\Classical\Classical\Jean-Baptiste Lully\Minuet.mid
Couldn't load : adl-piano-midi\Classical\Classical\

In [4]:
def rounded_accuracy(y_true, y_pred):
    y_pred_rounded = tf.round(y_pred)
    correct_predictions = tf.equal(tf.cast(y_pred_rounded, tf.int32), tf.cast(y_true, tf.int32))
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
    return accuracy

def tol_accuracy(y_true, y_pred):
    threshold = 0.1
    difference = tf.abs(tf.subtract(y_true, y_pred)) - threshold
    correct_predictions = tf.where(difference <= 0, True, False)
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
    return accuracy

def plot_training_results(title,loss,channels_accuracy,notes_accuracy,velocities_accuracy,times_accuracy,val_loss,val_channels_accuracy,val_notes_accuracy,val_velocities_accuracy,val_times_accuracy):
    n = len(loss)+1
    x = range(1,n)
    x_ticks = range(1,n,max(1,n//10))

    plt.figure()
    plt.plot(x, loss, label="Training loss")
    plt.plot(x, val_loss, label="Validation loss")
    plt.title(title)
    plt.legend()
    plt.xlabel("Number of epochs")
    plt.ylabel("Loss")
    plt.xticks(x_ticks)
    plt.show()

    fig, axs = plt.subplots(2, 2, figsize=(9, 8))

    axs[0,0].plot(x, channels_accuracy, label="Training set")
    axs[0,0].plot(x, val_channels_accuracy, label="Validation set")
    axs[0,0].legend()
    axs[0,0].set_ylim([0,1])
    axs[0,0].set_title("Channels")
    axs[0,0].set_xlabel("Number of epochs")
    axs[0,0].set_ylabel("Accuracy")
    axs[0,0].set_xticks(x_ticks)

    axs[0,1].plot(x, notes_accuracy, label="Training set")
    axs[0,1].plot(x, val_notes_accuracy, label="Validation set")
    axs[0,1].legend()
    axs[0,1].set_ylim([0,1])
    axs[0,1].set_title("Notes")
    axs[0,1].set_xlabel("Number of epochs")
    axs[0,1].set_ylabel("Accuracy")
    axs[0,1].set_xticks(x_ticks)

    axs[1,0].plot(x, velocities_accuracy, label="Training set")
    axs[1,0].plot(x, val_velocities_accuracy, label="Validation set")
    axs[1,0].legend()
    axs[1,0].set_ylim([0,1])
    axs[1,0].set_title("Velocity")
    axs[1,0].set_xlabel("Number of epochs")
    axs[1,0].set_ylabel("Accuracy")
    axs[1,0].set_xticks(x_ticks)

    axs[1,1].plot(x, times_accuracy, label="Training set")
    axs[1,1].plot(x, val_times_accuracy, label="Validation set")
    axs[1,1].legend()
    axs[1,1].set_ylim([0,1])
    axs[1,1].set_title("Time")
    axs[1,1].set_xlabel("Number of epochs")
    axs[1,1].set_ylabel("Accuracy")
    axs[1,1].set_xticks(x_ticks)

    fig.suptitle(title)
    fig.tight_layout(rect=[0, 0, 1, 1])

In [5]:
seq_length = 30
val_split = 0.1
test_split = 0.1
train_split = 1 - val_split - test_split

total_samples = sum((len(song) - 1) // seq_length for song in ClassicSongs.values())

X_Channels = np.zeros((total_samples, seq_length, n_Channels))
X_Notes = np.zeros((total_samples, seq_length, n_Notes))
X_Velocities = np.zeros((total_samples, seq_length, n_Velocities))
X_Times = np.zeros((total_samples, seq_length, 1))
y_Channels = np.zeros((total_samples, seq_length, n_Channels))
y_Notes = np.zeros((total_samples, seq_length, n_Notes))
y_Velocities = np.zeros((total_samples, seq_length, n_Velocities))
y_Times = np.zeros((total_samples, seq_length, 1))

current_index = 0
for song in ClassicSongs.values():
    song_x_channels, song_y_channels = pp.label_sequences(pp.one_hot_encode(channel_to_ind, song[:, 0]), seq_length)
    song_x_notes, song_y_notes = pp.label_sequences(pp.one_hot_encode(note_to_ind, song[:, 1]), seq_length)
    song_x_velocities, song_y_velocities = pp.label_sequences(pp.one_hot_encode(velocity_to_ind, song[:, 2]), seq_length)
    song_x_ticks, song_y_ticks = pp.label_sequences(song[:, 3:], seq_length)
    
    n_samples = song_x_channels.shape[0]
    next_index = current_index + n_samples
    
    X_Channels[current_index:next_index] = song_x_channels
    X_Notes[current_index:next_index] = song_x_notes
    X_Velocities[current_index:next_index] = song_x_velocities
    X_Times[current_index:next_index] = song_x_ticks
    y_Channels[current_index:next_index] = song_y_channels
    y_Notes[current_index:next_index] = song_y_notes
    y_Velocities[current_index:next_index] = song_y_velocities
    y_Times[current_index:next_index] = song_y_ticks
    
    current_index = next_index

n_val = int(total_samples*val_split)
n_test = int(total_samples*test_split)
indices = np.random.permutation(total_samples)
val_indices = indices[:n_val]
test_indices = indices[-n_test:]
train_indices = indices[n_val:-n_test]

Test_X_Channels = X_Channels[test_indices,:,:]
Test_X_Notes = X_Notes[test_indices,:,:]
Test_X_Velocities = X_Velocities[test_indices,:,:]
Test_X_Times = X_Times[test_indices,:,:]
Test_y_Channels = y_Channels[test_indices,:,:]
Test_y_Notes = y_Notes[test_indices,:,:]
Test_y_Velocities = y_Velocities[test_indices,:,:]
Test_y_Times = y_Times[test_indices,:,:]

Val_X_Channels = X_Channels[val_indices,:,:]
Val_X_Notes = X_Notes[val_indices,:,:]
Val_X_Velocities = X_Velocities[val_indices,:,:]
Val_X_Times = X_Times[val_indices,:,:]
Val_y_Channels = y_Channels[val_indices,:,:]
Val_y_Notes = y_Notes[val_indices,:,:]
Val_y_Velocities = y_Velocities[val_indices,:,:]
Val_y_Times = y_Times[val_indices,:,:]

X_Channels = X_Channels[train_indices,:,:]
X_Notes = X_Notes[train_indices,:,:]
X_Velocities = X_Velocities[train_indices,:,:]
X_Times = X_Times[train_indices,:,:]
y_Channels = y_Channels[train_indices,:,:]
y_Notes = y_Notes[train_indices,:,:]
y_Velocities = y_Velocities[train_indices,:,:]
y_Times = y_Times[train_indices,:,:]


del song_x_channels, song_y_channels, song_x_notes, song_y_notes, song_x_velocities, song_y_velocities, song_x_ticks, song_y_ticks
del dataset, ClassicSongs
gc.collect()

0

In [6]:
def create_model(modelType,dim):
    # Define input layer
    input_channels = Input(shape=(None, n_Channels))
    input_notes = Input(shape=(None, n_Notes))
    input_velocities = Input(shape=(None, n_Velocities))
    input_times = Input(shape=(None, 1))

    # Define main layer
    if modelType=="RNN" : output = SimpleRNN(units=dim, return_sequences=True)(concatenate([input_channels, input_notes, input_velocities, input_times]))
    elif modelType=="LSTM" : output = LSTM(units=dim, return_sequences=True)(concatenate([input_channels, input_notes, input_velocities, input_times]))
    elif modelType=="LSTM2":
        hidden = LSTM(units=dim[0], return_sequences=True)(concatenate([input_channels, input_notes, input_velocities, input_times]))
        output = LSTM(units=dim[1], return_sequences=True)(hidden)
        
    # Define Dense layer for each branch
    channels_output = Dense(units=n_Channels)(output)
    notes_output = Dense(units=n_Notes)(output)
    velocities_output = Dense(units=n_Velocities)(output)
    times_output = Dense(units=1)(output)

    # Use Lambda layer to split the output into two branches
    final_channels = Softmax(name="Channels")(channels_output)
    final_notes = Softmax(name="Notes")(notes_output)
    final_velocities = Softmax(name="Velocities")(velocities_output)
    final_times = Lambda(lambda x: (tf.sigmoid(x) * (time_range[1] - time_range[0]) + time_range[0]), name="Times")(times_output)

    # Define the model with inputs and outputs
    return Model(inputs=[input_channels, input_notes, input_velocities, input_times], outputs=[final_channels, final_notes, final_velocities, final_times], name=modelType+"_model")

In [8]:
dim = 250
n_batch = 200
learning_rate = 0.001

optimizer = optimizers.Adam(learning_rate=learning_rate)

lossesRNN = {
    'Channels': 'categorical_crossentropy',
    'Notes': 'categorical_crossentropy',
    'Velocities': 'categorical_crossentropy',
    'Times': 'mean_squared_error'
}


loss_weightsRNN = {
    'Channels': 0.25,
    'Notes': 5.0,
    'Velocities': 1.0,
    'Times': 1e-6
}

metricsRNN = {
    'Channels': 'accuracy',
    'Notes': 'accuracy',
    'Velocities': 'accuracy',
    'Times': tol_accuracy
}

RNN_model = create_model("RNN",dim)
RNN_model.summary()
RNN_model.compile(optimizer=optimizer, loss=lossesRNN, loss_weights=loss_weightsRNN, metrics=metricsRNN)

lossRNN = []
channels_accuracyRNN = []
notes_accuracyRNN = []
velocities_accuracyRNN = []
times_accuracyRNN = []

val_lossRNN = []
val_channels_accuracyRNN = []
val_notes_accuracyRNN = []
val_velocities_accuracyRNN = []
val_times_accuracyRNN = []

Model: "RNN_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 12)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 105)]  0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, None, 128)]  0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, None, 1)]    0           []                               
                                                                                          

In [9]:
n_epochs = 20

history = RNN_model.fit([X_Channels, X_Notes, X_Velocities, X_Times], [y_Channels, y_Notes, y_Velocities, y_Times], epochs=n_epochs, batch_size=n_batch, validation_data=([Val_X_Channels, Val_X_Notes, Val_X_Velocities, Val_X_Times], [Val_y_Channels, Val_y_Notes, Val_y_Velocities, Val_y_Times]))

lossRNN += history.history['loss']
channels_accuracyRNN += history.history['Channels_accuracy']
notes_accuracyRNN += history.history['Notes_accuracy']
velocities_accuracyRNN += history.history['Velocities_accuracy']
times_accuracyRNN += history.history['Times_tol_accuracy']

val_lossRNN += history.history['val_loss']
val_channels_accuracyRNN += history.history['val_Channels_accuracy']
val_notes_accuracyRNN += history.history['val_Notes_accuracy']
val_velocities_accuracyRNN += history.history['val_Velocities_accuracy']
val_times_accuracyRNN += history.history['val_Times_tol_accuracy']

Epoch 1/20
284/284 [==============================] - 40s 115ms/step - loss: 20.1945 - Channels_loss: 0.7177 - Notes_loss: 3.5298 - Velocities_loss: 2.3657 - Times_loss: 442.7570 - Channels_accuracy: 0.7091 - Notes_accuracy: 0.1069 - Velocities_accuracy: 0.5901 - Times_tol_accuracy: 9.3897e-06 - val_loss: 18.1624 - val_Channels_loss: 0.6102 - val_Notes_loss: 3.1710 - val_Velocities_loss: 2.1547 - val_Times_loss: 78.0706 - val_Channels_accuracy: 0.7287 - val_Notes_accuracy: 0.1583 - val_Velocities_accuracy: 0.6107 - val_Times_tol_accuracy: 1.8519e-05
Epoch 2/20
284/284 [==============================] - 31s 108ms/step - loss: 17.6984 - Channels_loss: 0.5661 - Notes_loss: 3.1036 - Velocities_loss: 2.0386 - Times_loss: 46.1974 - Channels_accuracy: 0.7383 - Notes_accuracy: 0.1732 - Velocities_accuracy: 0.6161 - Times_tol_accuracy: 6.7034e-05 - val_loss: 17.3625 - val_Channels_loss: 0.5341 - val_Notes_loss: 3.0559 - val_Velocities_loss: 1.9495 - val_Times_loss: 27.5461 - val_Channels_accura

Epoch 16/20
284/284 [==============================] - 30s 107ms/step - loss: 15.6173 - Channels_loss: 0.4649 - Notes_loss: 2.7637 - Velocities_loss: 1.6825 - Times_loss: 0.8064 - Channels_accuracy: 0.7830 - Notes_accuracy: 0.2451 - Velocities_accuracy: 0.6392 - Times_tol_accuracy: 0.2128 - val_loss: 15.7580 - val_Channels_loss: 0.4630 - val_Notes_loss: 2.7898 - val_Velocities_loss: 1.6931 - val_Times_loss: 0.8035 - val_Channels_accuracy: 0.7860 - val_Notes_accuracy: 0.2402 - val_Velocities_accuracy: 0.6385 - val_Times_tol_accuracy: 0.2172
Epoch 17/20
284/284 [==============================] - 30s 107ms/step - loss: 15.5629 - Channels_loss: 0.4634 - Notes_loss: 2.7534 - Velocities_loss: 1.6799 - Times_loss: 0.6814 - Channels_accuracy: 0.7841 - Notes_accuracy: 0.2478 - Velocities_accuracy: 0.6397 - Times_tol_accuracy: 0.2426 - val_loss: 15.7271 - val_Channels_loss: 0.4593 - val_Notes_loss: 2.7842 - val_Velocities_loss: 1.6912 - val_Times_loss: 0.6445 - val_Channels_accuracy: 0.7881 - va

In [10]:
%matplotlib qt
plot_training_results("Training our Multi-Input Multi-Output RNN",lossRNN,channels_accuracyRNN,notes_accuracyRNN,velocities_accuracyRNN,times_accuracyRNN,val_lossRNN,val_channels_accuracyRNN,val_notes_accuracyRNN,val_velocities_accuracyRNN,val_times_accuracyRNN)

In [13]:
dim = 250
n_batch = 200
learning_rate = 0.001

optimizerLSTM = optimizers.Adam(learning_rate=learning_rate)

lossesLSTM = {
    'Channels': 'categorical_crossentropy',
    'Notes': 'categorical_crossentropy',
    'Velocities': 'categorical_crossentropy',
    'Times': 'mean_squared_error'
}


loss_weightsLSTM = {
    'Channels': 0.5,
    'Notes': 5.0,
    'Velocities': 1.0,
    'Times': 2e-7
}

metricsLSTM = {
    'Channels': 'accuracy',
    'Notes': 'accuracy',
    'Velocities': 'accuracy',
    'Times': tol_accuracy
}

LSTM_model = create_model("LSTM",dim)
LSTM_model.summary()
LSTM_model.compile(optimizer=optimizerLSTM, loss=lossesLSTM, loss_weights=loss_weightsLSTM, metrics=metricsLSTM)

lossLSTM = []
channels_accuracyLSTM = []
notes_accuracyLSTM = []
velocities_accuracyLSTM = []
times_accuracyLSTM = []

val_lossLSTM = []
val_channels_accuracyLSTM = []
val_notes_accuracyLSTM = []
val_velocities_accuracyLSTM = []
val_times_accuracyLSTM = []

Model: "RNN_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, None, 12)]   0           []                               
                                                                                                  
 input_14 (InputLayer)          [(None, None, 105)]  0           []                               
                                                                                                  
 input_15 (InputLayer)          [(None, None, 128)]  0           []                               
                                                                                                  
 input_16 (InputLayer)          [(None, None, 1)]    0           []                               
                                                                                          

In [14]:
n_epochs = 20

history = LSTM_model.fit([X_Channels, X_Notes, X_Velocities, X_Times], [y_Channels, y_Notes, y_Velocities, y_Times], epochs=n_epochs, batch_size=n_batch, validation_data=([Val_X_Channels, Val_X_Notes, Val_X_Velocities, Val_X_Times], [Val_y_Channels, Val_y_Notes, Val_y_Velocities, Val_y_Times]))

lossLSTM += history.history['loss']
channels_accuracyLSTM += history.history['Channels_accuracy']
notes_accuracyLSTM += history.history['Notes_accuracy']
velocities_accuracyLSTM += history.history['Velocities_accuracy']
times_accuracyLSTM += history.history['Times_tol_accuracy']

val_lossLSTM += history.history['val_loss']
val_channels_accuracyLSTM += history.history['val_Channels_accuracy']
val_notes_accuracyLSTM += history.history['val_Notes_accuracy']
val_velocities_accuracyLSTM += history.history['val_Velocities_accuracy']
val_times_accuracyLSTM += history.history['val_Times_tol_accuracy']

Epoch 1/20
284/284 [==============================] - 87s 276ms/step - loss: 21.2431 - Channels_loss: 0.7414 - Notes_loss: 3.6728 - Velocities_loss: 2.5082 - Times_loss: 500.4359 - Channels_accuracy: 0.7154 - Notes_accuracy: 0.0853 - Velocities_accuracy: 0.5746 - Times_tol_accuracy: 3.9838e-04 - val_loss: 19.0489 - val_Channels_loss: 0.5890 - val_Notes_loss: 3.3047 - val_Velocities_loss: 2.2311 - val_Times_loss: 88.1786 - val_Channels_accuracy: 0.7339 - val_Notes_accuracy: 0.1254 - val_Velocities_accuracy: 0.6081 - val_Times_tol_accuracy: 0.0018
Epoch 2/20
284/284 [==============================] - 71s 250ms/step - loss: 18.3436 - Channels_loss: 0.5577 - Notes_loss: 3.1887 - Velocities_loss: 2.1213 - Times_loss: 45.7824 - Channels_accuracy: 0.7351 - Notes_accuracy: 0.1423 - Velocities_accuracy: 0.6118 - Times_tol_accuracy: 0.0175 - val_loss: 17.8676 - val_Channels_loss: 0.5318 - val_Notes_loss: 3.1139 - val_Velocities_loss: 2.0324 - val_Times_loss: 21.5356 - val_Channels_accuracy: 0.74

284/284 [==============================] - 69s 243ms/step - loss: 14.5939 - Channels_loss: 0.4064 - Notes_loss: 2.5516 - Velocities_loss: 1.6327 - Times_loss: 0.6298 - Channels_accuracy: 0.8100 - Notes_accuracy: 0.2991 - Velocities_accuracy: 0.6450 - Times_tol_accuracy: 0.2033 - val_loss: 14.7830 - val_Channels_loss: 0.4044 - val_Notes_loss: 2.5877 - val_Velocities_loss: 1.6422 - val_Times_loss: 0.5327 - val_Channels_accuracy: 0.8114 - val_Notes_accuracy: 0.2930 - val_Velocities_accuracy: 0.6456 - val_Times_tol_accuracy: 0.2108
Epoch 17/20
284/284 [==============================] - 68s 238ms/step - loss: 14.4716 - Channels_loss: 0.4040 - Notes_loss: 2.5286 - Velocities_loss: 1.6264 - Times_loss: 0.5756 - Channels_accuracy: 0.8111 - Notes_accuracy: 0.3059 - Velocities_accuracy: 0.6464 - Times_tol_accuracy: 0.2359 - val_loss: 14.6870 - val_Channels_loss: 0.4042 - val_Notes_loss: 2.5696 - val_Velocities_loss: 1.6368 - val_Times_loss: 0.5023 - val_Channels_accuracy: 0.8114 - val_Notes_accu

In [15]:
%matplotlib qt
plot_training_results("Training our Multi-Input Multi-Output LSTM",lossLSTM,channels_accuracyLSTM,notes_accuracyLSTM,velocities_accuracyLSTM,times_accuracyLSTM,val_lossLSTM,val_channels_accuracyLSTM,val_notes_accuracyLSTM,val_velocities_accuracyLSTM,val_times_accuracyLSTM)

In [16]:
dims = [200,200]
n_batch = 200
learning_rate = 0.001

optimizerLSTM2 = optimizers.Adam(learning_rate=learning_rate)

lossesLSTM2 = {
    'Channels': 'categorical_crossentropy',
    'Notes': 'categorical_crossentropy',
    'Velocities': 'categorical_crossentropy',
    'Times': 'mean_squared_error'
}


loss_weightsLSTM2 = {
    'Channels': 0.5,
    'Notes': 5.0,
    'Velocities': 1.0,
    'Times': 2e-7
}

metricsLSTM2 = {
    'Channels': 'accuracy',
    'Notes': 'accuracy',
    'Velocities': 'accuracy',
    'Times': tol_accuracy
}

LSTM2_model = create_model("LSTM2",dims)
LSTM2_model.summary()
LSTM2_model.compile(optimizer=optimizerLSTM2, loss=lossesLSTM2, loss_weights=loss_weightsLSTM2, metrics=metricsLSTM2)

lossLSTM2 = []
channels_accuracyLSTM2 = []
notes_accuracyLSTM2 = []
velocities_accuracyLSTM2 = []
times_accuracyLSTM2 = []

val_lossLSTM2 = []
val_channels_accuracyLSTM2 = []
val_notes_accuracyLSTM2 = []
val_velocities_accuracyLSTM2 = []
val_times_accuracyLSTM2 = []

Model: "RNN_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_17 (InputLayer)          [(None, None, 12)]   0           []                               
                                                                                                  
 input_18 (InputLayer)          [(None, None, 105)]  0           []                               
                                                                                                  
 input_19 (InputLayer)          [(None, None, 128)]  0           []                               
                                                                                                  
 input_20 (InputLayer)          [(None, None, 1)]    0           []                               
                                                                                          

In [18]:
n_epochs = 5

history = LSTM2_model.fit([X_Channels, X_Notes, X_Velocities, X_Times], [y_Channels, y_Notes, y_Velocities, y_Times], epochs=n_epochs, batch_size=n_batch, validation_data=([Val_X_Channels, Val_X_Notes, Val_X_Velocities, Val_X_Times], [Val_y_Channels, Val_y_Notes, Val_y_Velocities, Val_y_Times]))

lossLSTM2 += history.history['loss']
channels_accuracyLSTM2 += history.history['Channels_accuracy']
notes_accuracyLSTM2 += history.history['Notes_accuracy']
velocities_accuracyLSTM2 += history.history['Velocities_accuracy']
times_accuracyLSTM2 += history.history['Times_tol_accuracy']

val_lossLSTM2 += history.history['val_loss']
val_channels_accuracyLSTM2 += history.history['val_Channels_accuracy']
val_notes_accuracyLSTM2 += history.history['val_Notes_accuracy']
val_velocities_accuracyLSTM2 += history.history['val_Velocities_accuracy']
val_times_accuracyLSTM2 += history.history['val_Times_tol_accuracy']

Epoch 1/5
284/284 [==============================] - 150s 506ms/step - loss: 14.3693 - Channels_loss: 0.4022 - Notes_loss: 2.5024 - Velocities_loss: 1.6564 - Times_loss: 1.3818 - Channels_accuracy: 0.8159 - Notes_accuracy: 0.3115 - Velocities_accuracy: 0.6438 - Times_tol_accuracy: 0.0490 - val_loss: 14.5879 - val_Channels_loss: 0.4015 - val_Notes_loss: 2.5440 - val_Velocities_loss: 1.6673 - val_Times_loss: 1.4410 - val_Channels_accuracy: 0.8165 - val_Notes_accuracy: 0.3035 - val_Velocities_accuracy: 0.6428 - val_Times_tol_accuracy: 0.0469
Epoch 2/5
284/284 [==============================] - 123s 434ms/step - loss: 14.2312 - Channels_loss: 0.3992 - Notes_loss: 2.4766 - Velocities_loss: 1.6486 - Times_loss: 1.2042 - Channels_accuracy: 0.8177 - Notes_accuracy: 0.3191 - Velocities_accuracy: 0.6454 - Times_tol_accuracy: 0.0627 - val_loss: 14.4545 - val_Channels_loss: 0.3974 - val_Notes_loss: 2.5192 - val_Velocities_loss: 1.6600 - val_Times_loss: 1.0183 - val_Channels_accuracy: 0.8177 - val_

In [24]:
n = len(lossRNN)+1
x = range(1,n)
x_ticks = range(1,n,max(1,n//10))

plt.figure()
plt.plot(x, lossRNN, label="RNN Training loss",color='red')
plt.plot(x, val_lossRNN, label="RNN Validation loss",color='firebrick')
plt.plot(x, lossLSTM, label="LSTM Training loss",color='blue')
plt.plot(x, val_lossLSTM, label="LSTM Validation loss",color='mediumblue')
plt.plot(x, lossLSTM2, label="2L LSTM Training loss",color='lawngreen')
plt.plot(x, val_lossLSTM2, label="2L LSTM Validation loss",color='forestgreen')
plt.title("Comparison RNN - LSTM")
plt.legend()
plt.xlabel("Number of epochs")
plt.ylabel("Loss")
plt.xticks(x_ticks)
plt.show()

fig, axs = plt.subplots(2, 2, figsize=(9, 8))

axs[0,0].plot(x, channels_accuracyRNN, label="RNN Training set",color='red')
axs[0,0].plot(x, val_channels_accuracyRNN, label="RNN Validation set",color='firebrick')
axs[0,0].plot(x, channels_accuracyLSTM, label="LSTM Training set",color='blue')
axs[0,0].plot(x, val_channels_accuracyLSTM, label="LSTM Validation set",color='mediumblue')
axs[0,0].plot(x, channels_accuracyLSTM2, label="2L LSTM Training set",color='lawngreen')
axs[0,0].plot(x, val_channels_accuracyLSTM2, label="2L LSTM Validation set",color='forestgreen')
axs[0,0].legend()
#axs[0,0].set_ylim([0,1])
axs[0,0].set_title("Channels")
axs[0,0].set_xlabel("Number of epochs")
axs[0,0].set_ylabel("Accuracy")
axs[0,0].set_xticks(x_ticks)

axs[0,1].plot(x, notes_accuracyRNN, label="RNN Training set",color='red')
axs[0,1].plot(x, val_notes_accuracyRNN, label="RNN Validation set",color='firebrick')
axs[0,1].plot(x, notes_accuracyLSTM, label="LSTM Training set",color='blue')
axs[0,1].plot(x, val_notes_accuracyLSTM, label="LSTM Validation set",color='mediumblue')
axs[0,1].plot(x, notes_accuracyLSTM2, label="2L LSTM Training set",color='lawngreen')
axs[0,1].plot(x, val_notes_accuracyLSTM2, label="2L LSTM Validation set",color='forestgreen')
axs[0,1].legend()
#axs[0,1].set_ylim([0,1])
axs[0,1].set_title("Notes")
axs[0,1].set_xlabel("Number of epochs")
axs[0,1].set_ylabel("Accuracy")
axs[0,1].set_xticks(x_ticks)

axs[1,0].plot(x, velocities_accuracyRNN, label="RNN Training set",color='red')
axs[1,0].plot(x, val_velocities_accuracyRNN, label="RNN Validation set",color='firebrick')
axs[1,0].plot(x, velocities_accuracyLSTM, label="LSTM Training set",color='blue')
axs[1,0].plot(x, val_velocities_accuracyLSTM, label="LSTM Validation set",color='mediumblue')
axs[1,0].plot(x, velocities_accuracyLSTM2, label="2L LSTM Training set",color='lawngreen')
axs[1,0].plot(x, val_velocities_accuracyLSTM2, label="2L LSTM Validation set",color='forestgreen')
axs[1,0].legend()
#axs[1,0].set_ylim([0,1])
axs[1,0].set_title("Velocity")
axs[1,0].set_xlabel("Number of epochs")
axs[1,0].set_ylabel("Accuracy")
axs[1,0].set_xticks(x_ticks)

axs[1,1].plot(x, times_accuracyRNN, label="RNN Training set",color='red')
axs[1,1].plot(x, val_times_accuracyRNN, label="RNN Validation set",color='firebrick')
axs[1,1].plot(x, times_accuracyLSTM, label="LSTM Training set",color='blue')
axs[1,1].plot(x, val_times_accuracyLSTM, label="LSTM Validation set",color='mediumblue')
axs[1,1].plot(x, times_accuracyLSTM2, label="2L LSTM Training set",color='lawngreen')
axs[1,1].plot(x, val_times_accuracyLSTM2, label="2L LSTM Validation set",color='forestgreen')
axs[1,1].legend()
#axs[1,1].set_ylim([0,1])
axs[1,1].set_title("Time")
axs[1,1].set_xlabel("Number of epochs")
axs[1,1].set_ylabel("Accuracy")
axs[1,1].set_xticks(x_ticks)

fig.suptitle("Comparison RNN - LSTM")
fig.tight_layout(rect=[0, 0, 1, 1])

In [3]:
def sample_probas(vec, mode, T):
    output = []
    if mode == "Max":
        channels = np.zeros_like(vec[0])
        i = np.argmax(vec[0][0,0,:])
        channels[0,0,i] = 1
        output += [channels]
        
        notes = np.zeros_like(vec[1])
        i = np.argmax(vec[1][0,0,:])
        notes[0,0,i] = 1
        output += [notes]
    
    elif mode == "Original":
        p = vec[0][0,0,:]
        channels = np.zeros_like(vec[0])
        i = np.random.choice(range(len(p)),p=p)
        channels[0,0,i] = 1
        output += [channels]
        
        p = vec[1][0,0,:]
        notes = np.zeros_like(vec[1])
        i = np.random.choice(range(len(p)),p=p)
        notes[0,0,i] = 1
        output += [notes]
    
    elif mode == "SoftMax":
        q = np.log(vec[0][0,0,:])
        p = np.exp((q - np.max(q))/T)
        p = p/np.sum(p)
        channels = np.zeros_like(vec[0])
        i = np.random.choice(range(len(p)),p=p)
        channels[0,0,i] = 1
        output += [channels]
        
        q = np.log(vec[1][0,0,:])
        p = np.exp((q - np.max(q))/T)
        p = p/np.sum(p)
        notes = np.zeros_like(vec[1])
        i = np.random.choice(range(len(p)),p=p)
        notes[0,0,i] = 1
        output += [notes]
        
    elif mode == "NucleusSampling":
        p = vec[0][0,0,:]
        sorted_p = np.sort(p)[::-1]
        cum_p = np.cumsum(sorted_p)
        k = np.sum(cum_p < T) + 1
        indices = np.argsort(p)[:-k]
        p[indices] = 0
        i = np.random.choice(range(len(p)),p=p/cum_p[k-1])
        channels = np.zeros_like(vec[0])
        channels[0,0,i] = 1
        output += [channels]
        
        p = vec[1][0,0,:]
        sorted_p = np.sort(p)[::-1]
        cum_p = np.cumsum(sorted_p)
        k = np.sum(cum_p < T) + 1
        indices = np.argsort(p)[:-k]
        p[indices] = 0
        i = np.random.choice(range(len(p)),p=p/cum_p[k-1])
        notes = np.zeros_like(vec[1])
        notes[0,0,i] = 1
        output += [notes]
        
    output += [np.rint(vec[2]).astype(int)]
    output += [vec[3]]
    return output

def generate(model,input_vec,output_length,mode="Max",T=1,one_hot=False,reset=False):
    stateful = model.stateful
    model.stateful = True
    if reset : model.reset_states()
    temp_vec = model.predict(input_vec)
    for i in range(4):
        temp_vec[i] = temp_vec[i][:,-1:,:]
    temp_vec = sample_probas(temp_vec,mode,T)
    if one_hot :
        output_vec = temp_vec
    else :
        output_vec = [[ind_to_channel[np.argmax(temp_vec[0][0,0,:])],ind_to_note[np.argmax(temp_vec[1][0,0,:])],temp_vec[2][0,0,0],temp_vec[3][0,0,0]]]
    for i in range(output_length-1):
        temp_vec = model.predict(temp_vec)
        temp_vec = sample_probas(temp_vec,mode,T)
        if one_hot :
            for j in range(4):
                output_vec[j] = np.concatenate((output_vec[j],temp_vec[j]),axis=1)
        else :
            output_vec += [[ind_to_channel[np.argmax(temp_vec[0][0,0,:])],ind_to_note[np.argmax(temp_vec[1][0,0,:])],temp_vec[2][0,0,0],temp_vec[3][0,0,0]]]
    model.stateful = stateful
    return output_vec
    


#LSTM_model.predict([X_Channels[0:1,0:1,:], X_notes[0:1,0:1,:], X_velocities[0:1,0:1,:], X_times[0:1,0:1,:]])
gen = generate(RNN_model,[X_Channels[2:3,:,:], X_Notes[2:3,:,:], X_Velocities[2:3,:,:], X_Times[2:3,:,:]],30,mode="SoftMax",T=0.8)
gen

NameError: name 'RNN_model' is not defined

In [1]:
def create_midi(gen):
    channels = []
    notes = []
    velocities = []
    times = []
    for i in range(len(gen)):
        channels += [gen[i][0]]
        notes += [gen[i][1]]
        velocities += [gen[i][2]]
        times += [gen[i][3]]
    return channels, notes, velocities, times

In [2]:
channels,notes,velocities,times = create_midi(gen)
print(channels)

NameError: name 'gen' is not defined